# Making predictions with Logistic Regression Model

In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass  # To get the password without showing the input
password = getpass.getpass()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay






In [ ]:
#1.Create a query or queries to extract the information you think may be relevant for building the prediction model. 
#It should include some film features and some rental features. Use the data from 2005.
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''
select i.store_id, r.rental_date, f.film_id, f.title, f.description, f.length, p.amount from rental as r
right join inventory as i
on r.inventory_id = i.inventory_id
left join film as f
on i.film_id = f.film_id
left join payment as p 
on r.rental_id = p.rental_id
where YEAR(r.rental_date)=2005
;
'''

data = pd.read_sql_query(query, engine)
data


In [ ]:
#2.Create a query to get the total amount of rentals in June for each film.
query = '''
select count(f.film_id) as 'amount_rentals', f.film_id, f.title from rental as r
left join inventory as i
on r.inventory_id = i.inventory_id
left join film as f
on i.film_id = f.film_id
where MONTH(r.rental_date)=6 AND YEAR(r.rental_date)=2005 
group by f.film_id
order by amount_rentals DESC;
'''
rental_6 = pd.read_sql_query(query, engine)
rental_6



In [ ]:
#3.Create a query to get the total amount of rentals in July for each film.

query = '''
select count(f.film_id) as 'amount_rentals', f.film_id, f.title from rental as r
left join inventory as i
on r.inventory_id = i.inventory_id
left join film as f
on i.film_id = f.film_id
where MONTH(r.rental_date)=7 AND YEAR(r.rental_date)=2005 
group by f.film_id
order by amount_rentals DESC;
'''
rental_7 = pd.read_sql_query(query, engine)
rental_7

In [ ]:
#4.Create a new column containing (Yes/No) for each film whether or not the number of monthly rentals in July was bigger than in June.
#Your objective will be to predict this new column.


rental_6_7=pd.merge(left = rental_6, # LEFT TABLE
                                 right = rental_7, # RIGHT TABLE
                                 how = 'right', # TYPE OF JOIN
                                 left_on = "film_id", # COMMON COLUMN IN LEFT TABLE
                                 right_on= "film_id") # COMMON COLUMN IN RIGTH TABLE

#rental_6_7.rename(columns={'amount_retnals_x':'amount_rentals_june','amount_retnals_y':'amount_rentals_july'})
rental_6_7

In [ ]:
#Cleaning data
#Replacing missing values
rental_6_7=rental_6_7.drop('title_x', axis=1)
rental_6_7=rental_6_7.fillna(0)
rental_6_7.isna().sum()

In [ ]:
#Creating a new column containing 'yes', 'no'

def amount_films_comparison(x,y):
    if y > x:
        return 'Yes'
    else:
        return 'No'
    
new_column=list(map(amount_films_comparison, rental_6_7['amount_rentals_x'],rental_6_7['amount_rentals_y']))

In [ ]:
rental_6_7['high_rental_july']= new_column

In [ ]:
#5.Read the data into a Pandas dataframe.
rental_6_7


In [ ]:
rental_6_7['high_rental_july'].value_counts()

In [ ]:
data2=pd.merge(left = rental_6_7, # LEFT TABLE
                                 right = data, # RIGHT TABLE
                                 how = 'right', # TYPE OF JOIN
                                 left_on = "film_id", # COMMON COLUMN IN LEFT TABLE
                                 right_on= "film_id") # COMMON COLUMN IN RIGTH TABLE



In [ ]:
data2

In [ ]:
#Correlation between features
corr_matrix=data2.corr(method='pearson')
sns.heatmap(corr_matrix, annot=True)


In [ ]:
pd.crosstab(data2['length'],data2['amount_rentals_x'])



In [ ]:
sns.displot(x='length',data=data2, bins=5)


In [ ]:
sns.displot(x='amount_rentals_x',data=data2, bins=5)



In [ ]:
#6.Analyze extracted features and transform them. You may need to encode some categorical variables or scale numerical variables.
#X-y split

X=data2.loc[:,['amount_rentals_x', 'length', 'amount']]
y=data2['high_rental_july']

#Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

#Scaling variables with StandardScaler

transformer = StandardScaler().fit(X_train) 

X_train_normalized = transformer.transform(X_train)
X_test_normalized  = transformer.transform(X_test)

X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train.columns)
X_test_norm  = pd.DataFrame(X_test_normalized, columns=X_test.columns)


#The selected data doesn't have categorical features. Therefore, there's no need to apply OHE

In [ ]:
X_train_norm

In [ ]:
#7.Create a logistic regression model to predict this new column from the cleaned data.
log_reg=LogisticRegression(random_state=11,solver='lbfgs',max_iter=1000).fit(X_train_norm, y_train)

y_test_pred = log_reg.predict(X_test_norm)

print(y_test_pred)

In [ ]:
#8.Evaluate the results.

print(accuracy_score(y_test,y_test_pred))

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)

disp = ConfusionMatrixDisplay(cm_test,display_labels=log_reg.classes_)
disp.plot()
plt.show()

As the dataset has 952 values for 'Yes' and just 6 for 'No', the model predicted all 'Yes'.
There's a problem of class imbalance.